In [10]:
#Data Download directory
dir="../Dataset_Kaggle"


In [ ]:
#Imports
import numpy as np 
import pandas as pd 
import seaborn as sns
from matplotlib import pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import os 
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, r2_score,precision_score, recall_score, f1_score


In [12]:
#Frames

df_stops=pd.read_csv(dir+"/pit_stops.csv")
df_constructor_standing=pd.read_csv(dir+"/constructor_standings.csv")
df_races=pd.read_csv(dir+"/races.csv")
df_result=pd.read_csv(dir+"/results.csv")




In [13]:
#Pre-processing for merging


df_result.drop(columns=['time','milliseconds'],axis=1,inplace=True)
df_constructor_standing.drop(columns=['position','positionText','points'],axis=1,inplace=True)
df_races.drop(columns=['time'],axis=1,inplace=True)

In [14]:
#Merge Frames
first_merge_cols=["raceId","driverId"]
commmon_columns_CS_PT=["raceId",'constructorId']
commmon_columns_CS_PT_RC=["raceId"]
merged_CS_PT=pd.merge(df_stops,df_result,on=first_merge_cols,how="inner")
merged_CS_PT=pd.merge(merged_CS_PT,df_constructor_standing,on=commmon_columns_CS_PT,how="inner")
merged_CS_PT_RC=pd.merge(merged_CS_PT,df_races,on=commmon_columns_CS_PT_RC,how="inner")
print(merged_CS_PT.columns)
print(merged_CS_PT_RC.head())

Index(['raceId', 'driverId', 'stop', 'lap', 'time', 'duration', 'milliseconds',
       'resultId', 'constructorId', 'number', 'grid', 'position',
       'positionText', 'positionOrder', 'points', 'laps', 'fastestLap', 'rank',
       'fastestLapTime', 'fastestLapSpeed', 'statusId',
       'constructorStandingsId', 'wins'],
      dtype='object')
   raceId  driverId  stop  lap      time duration  milliseconds  resultId  \
0     841       153     1    1  17:05:23   26.898         26898     20789   
1     841        30     1    1  17:05:52   25.021         25021     20797   
2     841        17     1   11  17:20:48   23.426         23426     20783   
3     841         4     1   12  17:22:34   23.251         23251     20782   
4     841        13     1   13  17:24:10   23.842         23842     20785   

   constructorId number  ...  fp1_date fp1_time fp2_date  fp2_time  fp3_date  \
0              5     19  ...        \N       \N       \N        \N        \N   
1            131      7  ...   

In [15]:
print(merged_CS_PT_RC.head())

   raceId  driverId  stop  lap      time duration  milliseconds  resultId  \
0     841       153     1    1  17:05:23   26.898         26898     20789   
1     841        30     1    1  17:05:52   25.021         25021     20797   
2     841        17     1   11  17:20:48   23.426         23426     20783   
3     841         4     1   12  17:22:34   23.251         23251     20782   
4     841        13     1   13  17:24:10   23.842         23842     20785   

   constructorId number  ...  fp1_date fp1_time fp2_date  fp2_time  fp3_date  \
0              5     19  ...        \N       \N       \N        \N        \N   
1            131      7  ...        \N       \N       \N        \N        \N   
2              9      2  ...        \N       \N       \N        \N        \N   
3              6      5  ...        \N       \N       \N        \N        \N   
4              6      6  ...        \N       \N       \N        \N        \N   

   fp3_time quali_date quali_time sprint_date sprint_tim

In [16]:
#Data cleaning

df_cleaned=merged_CS_PT_RC.drop(columns=['duration'])


# nan_count = merged_CS_PT_RC.isna().sum()
# print(nan_count)

# rows_with_nan = merged_CS_PT_RC[merged_CS_PT_RC.isna().any(axis=1)]
# print(rows_with_nan)
df_cleaned=merged_CS_PT_RC.drop(columns=['constructorStandingsId','name','positionOrder','positionText','fastestLap','fastestLapTime','fastestLapSpeed','time','milliseconds','number','duration',"round","date","url","fp1_date","fp1_time","fp3_time","quali_date","fp2_date","fp2_time","fp3_date","quali_time","sprint_date","sprint_time"])
# nan_count = merged_CS_PT_RC.isna().sum()
# print(nan_count)
df_cleaned.replace('\\N', np.nan, inplace=True)
df_cleaned['position'] = pd.to_numeric(df_cleaned['position'], errors='coerce')
df_cleaned['rank'] = pd.to_numeric(df_cleaned['rank'], errors='coerce')

df_sampled=df_cleaned.sample(n=500,random_state=42)

# Assuming df is your DataFrame

# Print all rows containing NaN values
df_cleaned.dropna(inplace=True)
# print(df_cleaned.describe())
print(df_cleaned.head())
print(df_cleaned.shape)

   raceId  driverId  stop  lap  resultId  constructorId  grid  position  \
0     841       153     1    1     20789              5    12      11.0   
2     841        17     1   11     20783              9     3       5.0   
3     841         4     1   12     20782              6     5       4.0   
4     841        13     1   13     20785              6     8       7.0   
6     841        20     1   14     20779              9     1       1.0   

   points  laps  rank  statusId  wins  year  circuitId  
0     0.0    57    10        11     0  2011          1  
2    10.0    58     3         1     1  2011          1  
3    12.0    58     2         1     0  2011          1  
4     6.0    58     1         1     0  2011          1  
6    25.0    58     4         1     1  2011          1  
(10061, 15)


Visuilazing data

In [17]:
# sns.pairplot(df_sampled);

In [18]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import math

# num_plots = len(df_cleaned.columns)

# n_cols = math.ceil(num_plots / 2) 
# n_rows = 2



# fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*5), constrained_layout=True)

# axes = axes.flatten()

# for idx, col in enumerate(df_cleaned.columns):
#     sns.violinplot(data=df_sampled, y=col, ax=axes[idx])  
#     axes[idx].set_title(f'Violin Plot of {col}')  

# if num_plots < len(axes):
#     for i in range(num_plots, len(axes)):
#         fig.delaxes(axes[i]) 

# plt.show()



In [19]:

# correlation_matrix = df_cleaned.corr()
# plt.figure(figsize=(10,6))
# sns.heatmap(correlation_matrix,annot=True,cmap='seismic',linewidths=0.5)
# plt.show()

In [20]:
#EDA
# df_sampled=df_cleaned.sample(n=5000,random_state=42)



# plt.figure(figsize=(10, 6))

df_grouped = df_cleaned[df_cleaned['points'] > 0].groupby(['constructorId', 'laps']).mean().reset_index()
# for constructor_id in df_grouped['constructorId'].unique():
#     constructor_data = df_grouped[df_grouped['constructorId'] == constructor_id]
#     plt.scatter(constructor_data['year'], constructor_data['lap'], label=f'Constructor {constructor_id}')

# # Adding plot labels and title
# plt.title('Average Pit Stop Lap by Constructor')
# plt.xlabel('Year')
# plt.ylabel('Pit Lap')
# plt.legend(title='Constructor ID')
# plt.grid(True)



In [21]:
# for constructor_id in df_grouped['constructorId'].unique():
#     constructor_data = df_grouped[df_grouped['constructorId'] == constructor_id]
#     plt.scatter(constructor_data['year'], constructor_data['stop'], label=f'Constructor {constructor_id}')

# # Adding plot labels and title
# plt.title('Average number of pit stops by Constructor')
# plt.xlabel('Year')
# plt.ylabel('No of stops')
# plt.legend(title='Constructor ID')
# plt.grid(True)

In [22]:
df_cleaned=df_cleaned[df_cleaned['stop']<=8.5]
df_cleaned=df_cleaned[df_cleaned['points']<=25]

df_cleaned=df_cleaned[df_cleaned['points']>=10]


print(df_cleaned.describe())
print(df_cleaned.shape)

df_cleaned.to_csv('Processed_DATA_PITSTRAT.csv', index=False)


            raceId     driverId         stop         lap      resultId  \
count  2661.000000  2661.000000  2661.000000  2661.00000   2661.000000   
mean    977.502818   383.894401     1.728298    26.24239  23549.499436   
std      88.150864   406.975514     0.893283    14.59172   1705.699434   
min     841.000000     1.000000     1.000000     1.00000  20779.000000   
25%     897.000000     4.000000     1.000000    14.00000  22066.000000   
50%     970.000000    20.000000     2.000000    26.00000  23403.000000   
75%    1055.000000   830.000000     2.000000    36.00000  25050.000000   
max    1132.000000   857.000000     6.000000    70.00000  26509.000000   

       constructorId         grid     position       points         laps  \
count    2661.000000  2661.000000  2661.000000  2661.000000  2661.000000   
mean       51.306652     4.788425     3.086434    15.719278    60.416009   
std        66.965584     3.887704     1.395228     5.077345     8.664170   
min         1.000000     0.00

In [23]:
#Call the model

cuda
Epoch [1/80], Loss: 1.2219 Validation loss: 0.7194
Epoch [2/80], Loss: 0.6303 Validation loss: 0.5163
Epoch [3/80], Loss: 0.4642 Validation loss: 0.4251
Epoch [4/80], Loss: 0.4011 Validation loss: 0.3894
Epoch [5/80], Loss: 0.3801 Validation loss: 0.4093
Epoch [6/80], Loss: 0.3347 Validation loss: 0.3931
Epoch [7/80], Loss: 0.3370 Validation loss: 0.4041
Epoch [8/80], Loss: 0.3441 Validation loss: 0.4117
Epoch [9/80], Loss: 0.3220 Validation loss: 0.4310
Epoch [10/80], Loss: 0.3203 Validation loss: 0.4029
Epoch [11/80], Loss: 0.3090 Validation loss: 0.3816
Epoch [12/80], Loss: 0.3011 Validation loss: 0.3862
Epoch [13/80], Loss: 0.2925 Validation loss: 0.3797
Epoch [14/80], Loss: 0.2902 Validation loss: 0.3935
Epoch [15/80], Loss: 0.2819 Validation loss: 0.4040
Epoch [16/80], Loss: 0.2764 Validation loss: 0.3964
Epoch [17/80], Loss: 0.2954 Validation loss: 0.3923
Epoch [18/80], Loss: 0.2813 Validation loss: 0.3875
Epoch [19/80], Loss: 0.2745 Validation loss: 0.3885
Epoch [20/80], L

In [24]:
# Get the first batch from the train_loader
inputs, labels = next(iter(train_loader))

# Select the first sample (index 0) from the batch
input_sample = inputs[4].unsqueeze(0)  # Add batch dimension (from shape [features] to [1, features])
label_sample = labels[4].unsqueeze(0)  # Add batch dimension (from shape [num_classes] to [1, num_classes])

# Assuming the inputs are already on the correct device (e.g., 'cuda')
input_sample = input_sample.to(device)
label_sample = label_sample.to(device)

# Split the input sample into circuit_onehot, race_onehot, and points_wins
circuit_onehot = input_sample[:, :circuit_id_max]
race_onehot = input_sample[:, circuit_id_max:circuit_id_max + race_id_max]
points_wins = input_sample[:, circuit_id_max + race_id_max:]

# Pass the split input sample to the model
output_sample = net(circuit_onehot, race_onehot, points_wins)

# Output the model's prediction for the sample

# Apply Softmax to the output sample along dim=1 (class dimension)
softmax = nn.Softmax(dim=1)  
output_probabilities = softmax(output_sample)

# Round the probabilities to 1 decimal place
output_probabilities_rounded = torch.round(output_probabilities * 1000) / 1000

print( output_probabilities)


tensor([[3.1410e-07, 4.7018e-06, 6.3995e-02, 9.1298e-01, 2.2156e-02, 8.2111e-04,
         4.5191e-05, 2.4746e-07]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [25]:
def validate_model(model, test_loader, criterion):
    model.eval()
    val_loss = 0.0
    all_targets = []
    all_predictions = []
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            circuit_onehot = inputs[:, :circuit_id_max]
            race_onehot = inputs[:, circuit_id_max:circuit_id_max + race_id_max]
            points_wins = inputs[:, circuit_id_max + race_id_max:]
            
            outputs = model(circuit_onehot, race_onehot, points_wins)
            loss = criterion(outputs, targets.argmax(dim=1))
            val_loss += loss.item()
            
            # Collecting predictions and targets for metrics
            _, predicted = torch.max(outputs, 1)
            all_targets.extend(targets.argmax(dim=1).cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
    
    avg_val_loss = val_loss / len(test_loader)
    
    # Calculate accuracy
    macro_accuracy = accuracy_score(all_targets, all_predictions)
    micro_accuracy = accuracy_score(all_targets, all_predictions, normalize=True)
    
    # Calculate R² (though not typical for classification)
    r2 = r2_score(all_targets, all_predictions)

    # Calculate F1 Score
    f1_macro = f1_score(all_targets, all_predictions, average='macro')
    f1_micro = f1_score(all_targets, all_predictions, average='micro')

    print(f"Validation Loss: {avg_val_loss:.4f}, Macro Accuracy: {macro_accuracy:.4f}, Micro Accuracy: {micro_accuracy:.4f}, R²: {r2:.4f}, F1 Macro: {f1_macro:.4f}, F1 Micro: {f1_micro:.4f}")
    
    return avg_val_loss
validate_model(net,test_loader,criterion)



Validation Loss: 0.3960, Macro Accuracy: 0.8705, Micro Accuracy: 0.8705, R²: 0.7802, F1 Macro: 0.5301, F1 Micro: 0.8705


0.39601948681999655

In [26]:
#SVM

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Data preparation
input_data = df_cleaned[['circuitId', 'raceId', 'points', 'lap', 'position', 'laps']].values
target_data = df_cleaned['stop'].values

scaler = StandardScaler()
input_data[:, 2:] = scaler.fit_transform(input_data[:, 2:])

circuit_encoded = OneHotEncoder(sparse_output=False).fit_transform(input_data[:, 0].reshape(-1, 1))
race_encoded = OneHotEncoder(sparse_output=False).fit_transform(input_data[:, 1].reshape(-1, 1))

X = pd.DataFrame(
    torch.cat([
        torch.tensor(circuit_encoded, dtype=torch.float32),
        torch.tensor(race_encoded, dtype=torch.float32),
        torch.tensor(input_data[:, 2:], dtype=torch.float32)
    ], dim=1).numpy()
)

y_encoded = pd.get_dummies(target_data).values

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# SVM model
svm_model = SVC(kernel='rbf', probability=True)  

# Training the model
svm_model.fit(X_train, y_train.argmax(axis=1))

# Predictions
train_predictions = svm_model.predict_proba(X_train)
train_loss = log_loss(y_train, train_predictions)

test_predictions = svm_model.predict_proba(X_test)
test_loss = log_loss(y_test, test_predictions)

# Print losses
print(f'Training Set Loss: {train_loss:.4f}')
print(f'Test Set Loss: {test_loss:.4f}')

# Class predictions
train_pred_labels = svm_model.predict(X_train)
test_pred_labels = svm_model.predict(X_test)

# Accuracy
train_accuracy = accuracy_score(y_train.argmax(axis=1), train_pred_labels)
test_accuracy = accuracy_score(y_test.argmax(axis=1), test_pred_labels)

# Precision, Recall, F1
train_precision = precision_score(y_train.argmax(axis=1), train_pred_labels, average='macro')
test_precision = precision_score(y_test.argmax(axis=1), test_pred_labels, average='macro')

train_recall = recall_score(y_train.argmax(axis=1), train_pred_labels, average='macro')
test_recall = recall_score(y_test.argmax(axis=1), test_pred_labels, average='macro')

train_f1 = f1_score(y_train.argmax(axis=1), train_pred_labels, average='macro')
test_f1 = f1_score(y_test.argmax(axis=1), test_pred_labels, average='macro')
train_r2 = r2_score(y_train.argmax(axis=1), train_pred_labels)
test_r2 = r2_score(y_test.argmax(axis=1), test_pred_labels)

# Print metrics
print(f"Training Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}, R2: {train_r2:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f},R2:{test_r2:.4f}")

train_precision = precision_score(y_train.argmax(axis=1), train_pred_labels, average='micro')
test_precision = precision_score(y_test.argmax(axis=1), test_pred_labels, average='micro')

train_recall = recall_score(y_train.argmax(axis=1), train_pred_labels, average='micro')
test_recall = recall_score(y_test.argmax(axis=1), test_pred_labels, average='micro')

train_f1 = f1_score(y_train.argmax(axis=1), train_pred_labels, average='micro')
test_f1 = f1_score(y_test.argmax(axis=1), test_pred_labels, average='micro')
train_r2 = r2_score(y_train.argmax(axis=1), train_pred_labels)
test_r2 = r2_score(y_test.argmax(axis=1), test_pred_labels)

print(f"Training Accuracy: {train_accuracy:.4f}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, F1: {train_f1:.4f}, R2: {train_r2:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f},R2:{test_r2:.4f}")



Training Set Loss: 0.4433
Test Set Loss: 0.6268
Training Accuracy: 0.8031, Precision: 0.5415, Recall: 0.3757, F1: 0.3783, R2: 0.5661
Test Accuracy: 0.7523, Precision: 0.3331, Recall: 0.3269, F1: 0.3270,R2:0.5119
Training Accuracy: 0.8031, Precision: 0.8031, Recall: 0.8031, F1: 0.8031, R2: 0.5661
Test Accuracy: 0.7523, Precision: 0.7523, Recall: 0.7523, F1: 0.7523,R2:0.5119


In [27]:
# from torchviz import make_dot
# circuit_id_max = 10  # Example value
# race_id_max = 5      # Example value

# # Instantiate the model
# model = MLPModel(circuit_id_max, race_id_max)

# # Create random one-hot encoded input data
# circuit_onehot = torch.randn(1, circuit_id_max)
# race_onehot = torch.randn(1, race_id_max)
# points_wins = torch.randn(1, 4)

# # Forward pass through the model
# output = model(circuit_onehot, race_onehot, points_wins)

# # Visualize the computation graph
# make_dot(output, params=dict(list(model.named_parameters()))).render("mlp_model_graph", format="png")


